**INSTALLING DEPENDENCIES**

In [ ]:
import pandas as pd
import numpy as np
import  datetime
from datetime import datetime, timedelta
import seaborn as sns

# ! pip install mitoinstaller
# ! python -m mitoinstaller install
# ! pip install openpyxl
! pip install xgboost
# ! pip install knnmv
# ! pip install lightgbm
import xgboost as xg
# from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb
# from knnmv.impute import KNNMVImputer
# from sklearn.metrics import mean_squared_error

In [ ]:
# Tools for processing data
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix, make_scorer, adjusted_rand_score, silhouette_score, homogeneity_score, normalized_mutual_info_score
# Classifiers, supervised and unsupervised
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation
from sklearn.feature_extraction.text import CountVectorizer
from time import time
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import imageio
import re
from collections import Counter
import spacy
from time import time
%matplotlib inline

**OBTAINING DATA**

In [ ]:
pred_data = pd.read_csv('../input/stat440-21-project2/pred.csv')
test_data = pd.read_csv('../input/stat440-21-project2/Xte.csv')
train_data = pd.read_csv('../input/stat440-21-project2/XYtr.csv')
XYtr = pd.read_csv('../input/stat440-21-project2/XYtr.csv')
Xte  = pd.read_csv('../input/stat440-21-project2/Xte.csv')


**PREPARIG DATA**

In [ ]:
name = list(["XYtr","Xte"])
j = 0
for df in [XYtr,Xte]:
    fp = open(str(name[j]) + '_fi.csv', 'w')
    fp.write('id,fi1,fi2,fi3,fi4,fi5,fi6,fi7\n')
    for i in range(df.shape[0]):
        id = df.loc[i,'id']
        f = df.loc[i,'id'] + df.loc[i, 'ext']
        try:
            pic = imageio.imread('../input/stat440-21-project2/images/images/' + f)
            fi1 = pic.shape[0]
            fi2 = pic.shape[1]
            fi3 = np.mean(pic[:,:,:])
            fi4 = pic.min()
            fi5 = np.mean(pic[:,:,0])
            fi6 = np.mean(pic[:,:,1])
            fi7 = np.mean(pic[:,:,2])
    
        except:
            fi1 = np.nan
            fi2 = np.nan
            fi3 = np.nan
            fi4 = np.nan
            fi5 = np.nan
            fi6 = np.nan
            fi7 = np.nan
            pass
    
        fp.write('%s,%f,%f,%f,%f,%f,%f,%f\n' % (id, fi1,fi2,fi3,fi4,fi5,fi6,fi7))
        fp.close()  
        j = j + 1
XYtr_fi = pd.read_csv('./XYtr_fi.csv')
XYtr_fi1 = XYtr_fi.drop(['id'], axis = 1).copy()
XYtr_fi1 = XYtr_fi1.fillna(0)
Xte_fi = pd.read_csv('./Xte_fi.csv')
Xte_fi1 = Xte_fi.drop(['id'], axis = 1).copy()
Xte_fi1= Xte_fi1.fillna(0)
train_data = pd.concat([XYtr, XYtr_fi1], axis =1)
test_data = pd.concat([Xte, Xte_fi1], axis =1)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
# Make corpus and vocab

K = 10
XYtr['description'] = XYtr['description'].fillna("NAN")
Xte['description'] = Xte['description'].fillna("NAN")
corpus = list(XYtr['description'])+list(Xte['description'])
vectorizer = CountVectorizer()
corpus = vectorizer.fit_transform(corpus)
lda = LatentDirichletAllocation(n_components = K)
lda.fit(corpus)

topics = lda.transform(corpus)
N = XYtr.shape[0]

fp = open('XYtr_ft.csv', 'w')
fp.write('id')
for k in range(K):
    fp.write(',FT%04d' % k)

fp.write('\n')
for i in range(N):
    id = XYtr.loc[i,'id']
    fp.write('%s' % id)
    for k in range(K):
        fp.write(',%f' % topics[i, k])
    
    fp.write('\n')

fp.close()    
fp = open('Xte_ft.csv', 'w')
fp.write('id')
for k in range(K):
    fp.write(',FT%04d' % k)

fp.write('\n')
for i in range(N):
    id = Xte.loc[i,'id']
    fp.write('%s' % id)
    for k in range(K):
        fp.write(',%f' % topics[i + N, k])
    
    fp.write('\n')

fp.close()    

XYtr_ft = pd.read_csv('XYtr_ft.csv')
XYtr_ft1 = XYtr_ft.drop(['id'], axis = 1).copy()
XYtr_ft1 = XYtr_ft1.fillna(0)

Xte_ft = pd.read_csv('Xte_ft.csv')
Xte_ft1 = Xte_ft.drop(['id'], axis = 1).copy()
Xte_ft1= Xte_ft1.fillna(0)

train_data = pd.concat([train_data, XYtr_ft1], axis =1)
test_data = pd.concat([test_data, Xte_ft1], axis =1)
# combining test and train data for imputation
combined_data = pd.concat([train_data, test_data], ignore_index=True)
combined_data = combined_data.drop(['total'], axis = 1)